# El objetivo del feature engineering es introducir variables derivadas que capturen relaciones y señales de riesgo temprano que no son fácilmente detectadas por los modelos a partir de las variables originales.

In [16]:
import pandas as pd
import os
import sys

# Obtener ruta absoluta 
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from src.utils.constants import(
    VARS_BINARIAS,
    VARS_CATEGORICAS_NOMINALES,
    VARS_CATEGORICAS_ORDINALES,
    VARS_NUMERICAS,
    TARGET,
    TARGET_VALUES,
    LABELS    
)

In [17]:
# Cargar dataset preprocesado
df_preproc = pd.read_csv('../data/processed/preprocessed_data.csv')
df = df_preproc.copy()  

print(f"Dataset cargado: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\nTarget binario:")
print(df['target_binario'].value_counts())
print(f"\nRatio: {df['target_binario'].value_counts()[0] / df['target_binario'].value_counts()[1]:.2f}:1")

df.head()

Dataset cargado: 4424 filas x 38 columnas

Target binario:
target_binario
0    3003
1    1421
Name: count, dtype: int64

Ratio: 2.11:1


,application_order,course,daytimeevening_attendance,previous_qualification_grade,admission_grade,displaced,educational_special_needs,debtor,tuition_fees_up_to_date,gender,...,is_single,application_mode_risk,is_over_23_entry,previous_qualification_risk,mothers_qualification_level,fathers_qualification_level,mothers_occupation_level,fathers_occupation_level,has_unknown_parent_info,target_binario
0,5,171,1,122.0,127.3,1,0,0,1,1,...,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Secundaria,Otro_Trabajo,Otro_Trabajo,0,1
1,1,9254,1,160.0,142.5,1,0,0,0,1,...,1,Bajo_Riesgo,0,Bajo_Riesgo,Secundaria,Superior,Profesional,Profesional,0,0
2,5,9070,1,122.0,124.8,1,0,0,0,1,...,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Baja,Basica_Baja,Otro_Trabajo,Otro_Trabajo,0,1
3,2,9773,1,122.0,119.6,1,0,0,1,0,...,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Basica_Baja,Otro_Trabajo,Profesional,0,0
4,1,8014,0,100.0,141.5,0,0,0,1,0,...,0,Alto_Riesgo,1,Bajo_Riesgo,Basica_Baja,Basica_Media,Otro_Trabajo,Otro_Trabajo,0,0


## 1. VARIABLE 1: financial_risk_flag

In [18]:
"""
Lógica:
Estudiantes con deudas y sin beca presentan mayor presión financiera.
"""

df['financial_risk_flag'] = (
    (df['debtor'] == 1) &
    (df['scholarship_holder'] == 0)
).astype(int)

print(df['financial_risk_flag'].value_counts())
print("\nTasa de deserción:")
print(
    df.groupby('financial_risk_flag')['target_binario']
      .apply(lambda x: (x == 'Dropout').mean())
      .round(3)
)

financial_risk_flag
0    4005
1     419
Name: count, dtype: int64

Tasa de deserción:
financial_risk_flag
0    0.0
1    0.0
Name: target_binario, dtype: float64


## 2. VARIABLE 2: early_academic_failure_flag

In [8]:
"""
Hipótesis:
El fracaso académico temprano es uno de los predictores más fuertes
de abandono estudiantil.
"""

df['early_academic_failure_flag'] = (
    (df['curricular_units_1st_sem_enrolled'] > 0) &
    (df['curricular_units_1st_sem_approved'] == 0)
).astype(int)

print(df['early_academic_failure_flag'].value_counts())
print("\nTasa de deserción:")
print(
    df.groupby('early_academic_failure_flag')['target_binario']
      .apply(lambda x: (x == 'Dropout').mean())
      .round(3)
)

early_academic_failure_flag
0    3886
1     538
Name: count, dtype: int64

Tasa de deserción:
early_academic_failure_flag
0    0.0
1    0.0
Name: target_binario, dtype: float64


## 3. VARIABLE 3: efinancial_academic_crisis_flag

In [9]:
"""
Hipótesis:
La combinación de presión financiera elevada y fracaso académico temprano
representa un escenario crítico de riesgo de deserción.
"""

df['financial_pressure_score'] = (
    df['debtor'] +
    (1 - df['scholarship_holder']) +
    (1 - df['tuition_fees_up_to_date'])
)

df['financial_academic_crisis_flag'] = (
    (df['financial_pressure_score'] >= 2) &
    (df['early_academic_failure_flag'] == 1)
).astype(int)

print(df['financial_academic_crisis_flag'].value_counts())
print("\nTasa de deserción:")
print(
    df.groupby('financial_academic_crisis_flag')['target_binario']
      .apply(lambda x: (x == 'Dropout').mean())
      .round(3)
)

financial_academic_crisis_flag
0    4216
1     208
Name: count, dtype: int64

Tasa de deserción:
financial_academic_crisis_flag
0    0.0
1    0.0
Name: target_binario, dtype: float64


## 4.  Limpieza de variables intermedias

In [19]:
variables_a_eliminar = [
    "financial_pressure_score",
    "early_academic_failure_flag"
]

df = df.drop(
    columns=[c for c in variables_a_eliminar if c in df.columns]
)

print("Variables eliminadas:", variables_a_eliminar)

Variables eliminadas: ['financial_pressure_score', 'early_academic_failure_flag']


In [24]:
print("Preprocesamiento de variables nueva terminado")
df.head(5)


Preprocesamiento de variables nueva terminado


,application_order,course,daytimeevening_attendance,previous_qualification_grade,admission_grade,displaced,educational_special_needs,debtor,tuition_fees_up_to_date,gender,...,application_mode_risk,is_over_23_entry,previous_qualification_risk,mothers_qualification_level,fathers_qualification_level,mothers_occupation_level,fathers_occupation_level,has_unknown_parent_info,target_binario,financial_risk_flag
0,5,171,1,122.0,127.3,1,0,0,1,1,...,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Secundaria,Otro_Trabajo,Otro_Trabajo,0,1,0
1,1,9254,1,160.0,142.5,1,0,0,0,1,...,Bajo_Riesgo,0,Bajo_Riesgo,Secundaria,Superior,Profesional,Profesional,0,0,0
2,5,9070,1,122.0,124.8,1,0,0,0,1,...,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Baja,Basica_Baja,Otro_Trabajo,Otro_Trabajo,0,1,0
3,2,9773,1,122.0,119.6,1,0,0,1,0,...,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Basica_Baja,Otro_Trabajo,Profesional,0,0,0
4,1,8014,0,100.0,141.5,0,0,0,1,0,...,Alto_Riesgo,1,Bajo_Riesgo,Basica_Baja,Basica_Media,Otro_Trabajo,Otro_Trabajo,0,0,0


## 5. GUARDAR DF PROCESADO

In [ ]:
output_path = "../data/processed/data_with_feature_financial_risk_flag.csv"
df.to_csv(output_path, index=False)
print(f"Dataset procesado guardado en: {output_path}")

df.head()

Dataset con features guardado en: ../data/processed/data_with_feature_financial_risk_flag.csv
(4424, 39)


## 6. Conclusiones


- Se diseñaron 5 variables derivadas basadas en hipótesis académicas,   socioeconómicas y académicas tempranas.
- Las features fueron evaluadas individualmente y se evaluó su incorporación final al dataset
- El impacto marginal sobre el recall fue limitado, lo que sugiere que las variables no aportan información adicional al modelo en consecuencia no serán incluídas.